# **Introduction**
This notebook provides an end-to-end pipeline for summarizing both English and Arabic books. It involves extracting text from a PDF, dividing the text into semantically coherent chunks, summarizing each chunk, and finally generating a PDF output of the summary. The pipeline automatically detects the language of the book and applies the appropriate summarization model. The summarization models used are optimized to run on GPUs, ensuring efficiency for large texts.

The steps for the pipeline include:

1. **Text Extraction**: Extract the raw text from the PDF file.
2. **Language Detection**: Detect whether the text is in English or Arabic.
3. **Semantic Chunking**: Break the text into semantically meaningful chunks based on sentence embeddings or natural chunking (depending on the language).
4. **Text Summarization**: Summarize each chunk using the appropriate model (BART for English, mT5 for Arabic).
5. **PDF Generation**: Create a PDF file containing the summarized text.


# **Pipeline Steps**


## Step 1: Install the Required Libraries


In [1]:
!pip install PyMuPDF pdfplumber transformers arabic_reshaper python-bidi matplotlib reportlab fpdf2
!pip install spacy camel-tools sentence-transformers fpdf PyPDF2 stanza langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.4/206.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2

## Step 2: Import Required Libraries


In [2]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import re
from fpdf import FPDF
import shutil
import pdfplumber
import arabic_reshaper
from bidi.algorithm import get_display
import stanza
from langdetect import detect


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Step 3: Download the Arabic Language Model for Stanza


In [3]:
stanza.download('ar')


## Step 4: Define Helper Functions for Text Extraction and Cleaning


In [35]:
# Extract text from the PDF file using pdfplumber
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''.join(page.extract_text() for page in pdf.pages)
    return text

# Function: Arabic Text Reshaping and Bidi Fix
def fix_arabic_text(text):
    reshaped_text = arabic_reshaper.reshape(text)
    return get_display(reshaped_text)


# Clean the text by removing URLs, numbers, and extra spaces
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b[A-Za-z]\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


## Step 5: Set Up the Sentence-BERT Model and Summarizer Models


In [36]:
# Load pre-trained Sentence-BERT model for semantic embeddings (ensure GPU usage)
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
summarizer_en = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
summarizer_ar = pipeline('summarization', model='csebuetnlp/mT5_multilingual_XLSum', device=0)
nlp_ar = stanza.Pipeline('ar', processors='tokenize')


## Step 6: Define the Function for Semantic Chunking in English


In [37]:
def divide_by_semantics_with_length(text, threshold=0.6, max_words=800, min_words=400):
    sentences = text.split('. ')
    embeddings = model.encode(sentences, convert_to_tensor=True)
    chunks = []
    current_chunk = sentences[0]

    for i in range(1, len(sentences)):
        similarity = util.pytorch_cos_sim(embeddings[i], embeddings[i - 1])
        current_word_count = len(current_chunk.split())

        if similarity < threshold or current_word_count + len(sentences[i].split()) > max_words:
            if current_word_count >= min_words:
                chunks.append(current_chunk.strip())
                current_chunk = sentences[i]
            else:
                current_chunk += '. ' + sentences[i]
        else:
            current_chunk += '. ' + sentences[i]

    if len(current_chunk.split()) >= min_words:
        chunks.append(current_chunk.strip())

    return chunks


## Step 7: Define the Function for Semantic Chunking in Arabic

In [38]:
# Function: Semantic Chunking (Arabic)
def chunk_arabic_text(text, min_words=300, max_words=500):
    """Break the Arabic text into semantically meaningful chunks."""
    doc = nlp_ar(text)
    chunks = []
    current_chunk = []
    current_chunk_word_count = 0

    for sentence in doc.sentences:
        sentence_text = sentence.text
        sentence_word_count = len(sentence_text.split())

        # If the sentence is too long, split it into smaller sentences
        if sentence_word_count > max_words:
            split_sentences = split_long_sentence(sentence_text, max_words)
        else:
            split_sentences = [sentence_text]

        # Add the split sentences to the current chunk
        for split_sentence in split_sentences:
            split_sentence_word_count = len(split_sentence.split())
            if current_chunk_word_count + split_sentence_word_count > max_words and current_chunk_word_count >= min_words:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_chunk_word_count = 0

            current_chunk.append(split_sentence)
            current_chunk_word_count += split_sentence_word_count

    # Add the last chunk if it meets the minimum word requirement
    if current_chunk_word_count >= min_words:
        chunks.append(' '.join(current_chunk))

    return chunks

# Helper function to split long Arabic sentences
def split_long_sentence(sentence_text, max_words):
    words = sentence_text.split()
    return [' '.join(words[i:i + max_words]) for i in range(0, len(words), max_words)]


## Step 8: Define the Summarization and PDF Generation Functions


In [56]:
# Summarize text chunks
def summarize_chunks(chunks, summarizer, min_chunk_length=50, max_summary_length=300, min_summary_length=80):
    summaries = []
    for chunk in chunks:
        if len(chunk.split()) > min_chunk_length:
            try:
                summary = summarizer(chunk, max_length=max_summary_length, min_length=min_summary_length, do_sample=False)[0]['summary_text']
                summaries.append(summary)
            except Exception as e:
                print(f"Error summarizing chunk: {e}")
                summaries.append(chunk)
        else:
            summaries.append(chunk)
    return summaries

# Generate PDF with the summarized text

reshaped_text = arabic_reshaper.reshape("ملخص الكتاب")
_text = get_display(reshaped_text)


class PDF(FPDF):
    def header(self):
        if self.page_no() == 1:
            self.set_font('DejaVu', 'B', 12)
            title = _text if self.language == 'ar' else 'Book Summary'
            self.cell(0, 10, title, ln=True, align='C')
            self.ln(10)

    def chapter_body(self, body):
        self.set_font('DejaVu', '', 12)
        self.multi_cell(0, 10, body, align='R' if self.language == 'ar' else 'L')
        self.ln()

    def add_text(self, text):
        self.add_page()
        self.chapter_body(text)

# Helper function to generate the PDF
def generate_pdf(summary_text, pdf_output_path, language='en'):
    pdf = PDF()
    pdf.language = language
    pdf.add_font('DejaVu', '', '/path/to/DejaVuSans.ttf', uni=True)
    pdf.add_font('DejaVu', 'B', '/path/to/DejaVuSans-Bold.ttf', uni=True)
    pdf.set_font('DejaVu', '', 12)

    pdf.add_text(summary_text)
    pdf.output(pdf_output_path)
    


## Step 9: Define the Summarization Pipelines for English and Arabic


### **English Summarization Pipeline**


In [57]:
def summarize_english(book_text, pdf_output_path="english_summary.pdf"):
    # Step 1: Divide text into semantic chunks
    semantic_chunks = divide_by_semantics_with_length(book_text)

    # Step 2: Clean the chunks
    cleaned_chunks = [clean_text(chunk) for chunk in semantic_chunks]

    # Step 3: Summarize the chunks
    summarized_chunks = summarize_chunks(cleaned_chunks, summarizer_en)

    # Step 4: Generate PDF
    final_summary = '\n\n'.join(summarized_chunks)
    generate_pdf(final_summary, pdf_output_path, language='en')

    print(f"Summarization completed!, saved to {pdf_output_path}")
    
    return final_summary


### **Arabic Summarization Pipeline**


In [58]:
def summarize_arabic(pdf_path, pdf_output_path="arabic_summary.pdf"):
    # Step 1: Extract text from PDF and fix Arabic text direction
    text = extract_text_from_pdf(pdf_path)
    fixed_text = fix_arabic_text(text)  # Fixing the text direction
    
    # Step 2: Chunk the text semantically
    chunks = chunk_arabic_text(fixed_text)  # Now the chunking function is defined
    
    # Step 3: Summarize the chunks
    summarized_chunks = summarize_chunks(chunks, summarizer_ar)
    
    # Step 4: Clean and generate the final summary
    cleaned_summaries = [clean_text(chunk) for chunk in summarized_chunks]
    final_summary = '\n\n'.join(cleaned_summaries)
    
    # Step 5: Generate PDF
    final_summary_arabic = fix_arabic_text(final_summary)
    generate_pdf(final_summary_arabic, pdf_output_path, language='ar')

    # Notify the user that the PDF has been created
    print(f"Summarization completed!, saved to {pdf_output_path}")

    return final_summary


## Step 10: Language Detection and Pipeline Execution


In [59]:
def detect_language_and_summarize(pdf_path, pdf_output_path_ar="arabic_summary.pdf", pdf_output_path_en="english_summary.pdf"):
    text = extract_text_from_pdf(pdf_path)
    language = detect(text)

    if language == 'ar':
        print("Detected Arabic. Running Arabic summarization pipeline...")
        return summarize_arabic(pdf_path, pdf_output_path=pdf_output_path_ar)
    else:
        print("Detected English. Running English summarization pipeline...")
        return summarize_english(text, pdf_output_path=pdf_output_path_en)


## Step 11: Run the Pipeline on Your PDF


In [55]:
pdf_path = "/path/to/your/book.pdf"  # Update this to the correct PDF path
final_summary = detect_language_and_summarize(pdf_path)


Detected Arabic. Running Arabic summarization pipeline...


/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -64475
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65115
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65118
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65122
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65123
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -65125
  warnings.warn("cmap value too big/small: %s" % cm)
/opt/conda/lib/python3.10/site-packages/

Summarization completed!, saved to arabic_summary.pdf
